In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [12]:
# For Chrome
driver = webdriver.Firefox()

In [13]:
driver.get('https://x.com/')

In [24]:
import re

def extract_number_from_text(text):
    """
    Extracts the first occurrence of a number (Arabic or Western numerals) from the given text.
    """
    # Pattern to match Arabic and Western numerals
    pattern = r'[\d\u0660-\u0669]+'
    match = re.search(pattern, text)
    if match:
        num_str = match.group()
        # Convert Arabic numerals to Western numerals if necessary
        arabic_to_western = str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789')
        num_str = num_str.translate(arabic_to_western)
        return num_str
    else:
        return '0'
    

# Use the function to scrape tweets
list_of_start_dates = [ 
"2023-01-01", "2023-02-01", "2023-03-01","2023-04-01", "2023-05-01", "2023-06-01", "2023-07-01", "2023-08-01", "2023-09-01", "2023-10-01", "2023-11-01", "2023-12-01"
]

list_of_end_dates = [
"2023-01-30", "2023-02-30", "2023-03-30", "2023-04-30", "2023-05-30", "2023-06-30","2023-07-30", "2023-08-30", "2023-09-30", "2023-10-30", "2023-11-30", "2023-12-30"
]
list_of_hashtag_1 =[
'الذكاء_الاصطناعي_في_التعليم', 'الذكاء_الاصطناعى_في_التعليم', 'الذكاء_الاصطناعي', 


]

list_of_hashtag_2 = [
     'الذكاء_التوليدي', 'الذكاء_التوليدي_في_التعليم', 'الذكاءالاصطناعي', 'بالذكاء_الاصطناعي', 'الذكاء_الاصطناعى'


]

list_of_hashtag_3 = ['ذكاء_اصطناعي', 'ذكاء_التعليم', 'ذكاء_توليدي', 'بالذكاء_التوليدي', 'الذكاءالتوليدي']

all_data = {
    'tweet': [],
    'hashtags': [],
    'timestamp': [],
    'full_url': [],
    'like_count': [],
    'retweet_count': [],
}

print(len(list_of_start_dates))
print(len(list_of_end_dates))
print(len(list_of_hashtag_1))
print(len(list_of_hashtag_2))
print(len(list_of_hashtag_3))
print(len(all_data))
print(len(all_data['tweet']))



12
12
3
5
5
6
0


In [25]:
def get_tweet_data(driver, all_data):
    try:
        tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
        # print(len(tweets))
        for tweet in tweets:
            content_element = tweet.find_element(By.XPATH, './/div[@lang]')
            content = content_element.text

            hashtags_elements = content_element.find_elements(By.XPATH, './/a[contains(@href, "/hashtag/")]')
            hashtags = [hashtag.text for hashtag in hashtags_elements]
            
            timestamp = tweet.find_element(By.TAG_NAME, 'time').get_attribute('datetime')

            like_button = tweet.find_elements(By.XPATH, './/button[@data-testid="like"]')[0]
            aria_label = like_button.get_attribute('aria-label')
            like_count = extract_number_from_text(aria_label)
            
            retweet_button = tweet.find_elements(By.XPATH, './/button[@data-testid="retweet"]')[0]
            aria_label = retweet_button.get_attribute('aria-label')
            retweet_count = extract_number_from_text(aria_label)

            try:
                tweet_link = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/') and @role='link']")
                href = tweet_link.get_attribute('href')
                if href.startswith('/'):
                    full_url = 'https://x.com' + href
                else:
                    full_url = href
                # print(f'Tweet URL: {full_url}')
            except:
                full_url = ''

            all_data['tweet'].append(content)
            all_data['hashtags'].append(hashtags)
            all_data['like_count'].append(like_count)
            all_data['retweet_count'].append(retweet_count)
            all_data['timestamp'].append(timestamp)
            all_data['full_url'].append(full_url)
    except Exception as e:
        pass
    
    return True

In [26]:
def scrape_tweets(all_data, driver, hashtag, start_date, end_date, geocode="24.7136,46.6753,10km", max_tweets=100):
    # Construct the URL with parameters
    url = f"https://x.com/search?q=%23{hashtag}%20since%3A{start_date}%20until%3A{end_date}&src=typed_query&f=live&geocode={geocode}"
    driver.get(url)
    time.sleep(3)  # Allow the page to load
    tweet_data = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    i = 0

    while True:
        get_tweet_data(driver, all_data)
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(20)
        # Calculate new scroll height and compare with last height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        i+=1
        if i==10000:
            break

    print(len(all_data['tweet']))
    return True

In [ ]:
# for hashtag in list_of_hashtag_3:

all_data = {
    'tweet': [],
    'hashtags': [],
    'timestamp': [],
    'full_url': [],
    'like_count': [],
    'retweet_count': [],
}

print(len(list_of_start_dates))
print(len(list_of_end_dates))
print(len(list_of_hashtag_1))
print(len(list_of_hashtag_2))
print(len(list_of_hashtag_3))
print(len(all_data))
print(len(all_data['tweet']))

print('='*50)
scrape_tweets(all_data, driver, 'الذكاء_الاصطناعي', "2023-01-01", "2023-03-30", geocode="24.774265, 46.738586,10km", max_tweets=100)
# time.sleep(60)

12
12
3
5
5
6
0


In [95]:

print(len(all_data['tweet']))
print(len(all_data['hashtags']))
print(len(all_data['like_count']))
print(len(all_data['retweet_count']))
print(len(all_data['timestamp']))
print(len(all_data['full_url']))

0
0
0
0
0
0


In [96]:
import pandas as pd
df = pd.DataFrame(all_data)
df.head()

,tweet,hashtags,timestamp,full_url,like_count,retweet_count


In [97]:
df.to_csv('data/الذكاء_الاصطناعي/file11.csv', index=False)

In [21]:
df_ = (list(pd.read_csv('file1.csv')['tweet']))
len(df_)

FileNotFoundError: [Errno 2] No such file or directory: 'file1.csv'

In [ ]:
for i in range(2, 13):
    df_ += (list(pd.read_csv('file' + str(i) + '.csv')['tweet']))
    print(len(df_))

612
923
1242
1561
1830
2143
2395
2714
2958


In [114]:
len(set(df_))

2151

In [8]:
url = "https://x.com/search?q=%23الذكاء_الاصطناعى%20since%3A2023-01-01%20until%3A2023-01-31&src=typed_query&f=live&geocode=24.7136,46.6753,1000km"
driver.get(url)

In [26]:
tweets = all_data['tweet']
print(type(tweets))
print(len(tweets))
print(len(set(tweets)))

<class 'list'>
74
68


In [5]:
driver.get("https://x.com/search?q=%23الذكاء_الاصطناعى%20since%3A2023-01-01%20until%3A2023-01-31&src=typed_query&f=live&geocode=24.7136,46.6753,1000km"
)
time.sleep(3)

In [7]:
tweet_data = []
last_height = driver.execute_script("return document.body.scrollHeight")

i = 0

while True:
    # get_tweet_data(driver, all_data)
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    # Calculate new scroll height and compare with last height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    i+=1
    if i==2:
        break

tweets = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')
len(tweets)

3

In [8]:
import re
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

for tweet in tweets:
    try:
        # Extract tweet text
        content_element = tweet.find_element(By.XPATH, './/div[@lang]')
        content = content_element.text

        # Extract hashtags
        hashtags_elements = content_element.find_elements(By.XPATH, './/a[contains(@href, '/hashtag/')]')
        hashtags = [hashtag.text for hashtag in hashtags_elements]

        # Extract number of likes
        try:
            like_button = tweet.find_element(By.XPATH, './/button[@data-testid="like"]')
            aria_label = like_button.get_attribute('aria-label')
            match = re.search(r'^(\d+)', aria_label)
            like_count = match.group(1) if match else '0'
        except Exception as e:
            like_count = '0'
            print(f'Error extracting like count: {e}')

        # Extract number of retweets
        try:
            retweet_button = tweet.find_element(By.XPATH, './/button[@data-testid="retweet"]')
            aria_label = retweet_button.get_attribute('aria-label')
            match = re.search(r'^(\d+)', aria_label)
            retweet_count = match.group(1) if match else '0'
        except Exception as e:
            retweet_count = '0'
            print(f'Error extracting retweet count: {e}')

        # Extract the tweet URL
        try:
            tweet_link = tweet.find_element(By.XPATH, ".//a[contains(@href, '/status/') and @role='link']")
            href = tweet_link.get_attribute('href')
            if href.startswith('/'):
                full_url = 'https://x.com' + href
            else:
                full_url = href
            print(f'Tweet URL: {full_url}')
        except NoSuchElementException:
            full_url = None
            print('Tweet URL not found.')

        # Print the extracted data
        print(f'وسمُ التَّغريدة: {", ".join(hashtags)}')
        print(f'نصُّ التَّغريدة: {content}')
        print(f'عددُ مرَّات إعادة التَّغريدة: {retweet_count}')
        print(f'عددُ مرَّات الاعجاب بالتَّغريدة: {like_count}')
        print('-' * 80)

    except Exception as e:
        print(f'Error processing tweet: {e}')


Error processing tweet: name 'hashtag' is not defined
Error processing tweet: name 'hashtag' is not defined
Error processing tweet: name 'hashtag' is not defined


وسمُ التَّغريدة: #الذكاء_الاصطناعي, #الطب
نصُّ التَّغريدة: تم اختبار بروتين مضاد للميكروبات مصمم ب #الذكاء_الاصطناعي  في الحياة الواقعية وثبت أنه يعمل. 
ProGen تم تدريبه على الأحماض الأمينية ل 280 مليون بروتين. تفاعل معها  بطرق تشير إلى أنها يمكن أن تكون فعالة في قتل البكتيريا. يمكن استخدام نفس النهج لإنشاء أدوية جديدة. 
#الطب
عددُ مرَّات إعادة التَّغريدة: 2
عددُ مرَّات الاعجاب بالتَّغريدة: 3
--------------------------------------------------------------------------------
وسمُ التَّغريدة: #الذكاء_الاصطناعي
نصُّ التَّغريدة: تحول واضح من قبل كبرى شركات الاعلام للانتقال للذكاء الاصطناعي!

اليوم شركة CNET المختصة باخبار التقنية 
كشفت انها اعتمدت على #الذكاء_الاصطناعي لكتابة 77 مقالة منذ نوفمبر الماضي وهذا يعني 1% من اجمالي المقالات!
عددُ مرَّات إعادة التَّغريدة: 0
عددُ مرَّات الاعجاب بالتَّغريدة: 1
--------------------------------------------------------------------------------
وسمُ التَّغريدة: #تقارير_ميدار, #جوجل, #الذكاء_الاصطناعي
نصُّ التَّغريدة: #تقارير_ميدار

#جوجل تحول النصوص والأصو

In [8]:
!pip install "unstructured[all-docs]"

  Obtaining dependency information for unstructured[all-docs] from https://files.pythonhosted.org/packages/fa/60/0f628f29779c1c88bfb930eee39460819e3cc868a017dda35c5717ce3f67/unstructured-0.16.5-py3-none-any.whl.metadata
  Using cached unstructured-0.16.5-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/ef/56/4ddf8b36aa4b52077045b17ffb8958f3360b250df4143d1482d9d5bb54d5/emoji-2.14.0-py3-none-any.whl.metadata
  Using cached emoji-2.14.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Obtaining dependency information for backoff from https://files.pythonhosted.org/packages/df/73/b6e24bd22e6720ca8ee9a85a0c4a2971af8497d8f3193fa0539

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ar3935\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\~ydantic_core\\_pydantic_core.cp311-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\ar3935\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
from unstructured.partition.pdf import partition_pdf

ModuleNotFoundError: No module named 'unstructured'

ModuleNotFoundError: No module named 'partition_pdf'

In [9]:
!pip install arabic-reshaper
!pip install python-bidi


  Obtaining dependency information for arabic-reshaper from https://files.pythonhosted.org/packages/44/fb/e20b45d81d74d810b01bff408baf8af04abf1d55a1a289c8395ad0919a7c/arabic_reshaper-3.0.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\ar3935\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Obtaining dependency information for python-bidi from https://files.pythonhosted.org/packages/f5/44/1fcb4350e6283a1ccd64666373274dd76eb0a9f91c3c1ac8e06e5046c5a5/python_bidi-0.6.3-cp311-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/157.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/157.2 kB ? eta -:--:--
   ------- ------------------------------- 30.7/157.2 kB 660.6 kB/s eta 0:00:01
   -------------------------- ------------- 102.4/157.2 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 157.2/157.2 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\ar3935\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import arabic_reshaper
from bidi.algorithm import get_display

# Your extracted text
text = """ـه1433/5/17:ه ُخيرات 33252287 : ِّك َّصلا ُم ْقَر ةمكحم نم قيدصتلا رارق مقر 960 :ىوعدلا ُم ْقَر ـه1434/1/6 :هخيرات 344533: فانئتسلاا   قــيرط نــع بــيعلا رــيدقت - بــيعلا دوــجول عــيبلا خــسف - ةرايــس عــيب   فــلح - نــيعادتلما ىوــقأ بــناج في عرــشت نــميلا - ءارــبلخا مــسق   نــب راــيلخا ىرتــشمللف بــيعلا دوــجو دــنع - راهظتــسلاا نــيم ىــعدلما   ميلــست هــيلع ىــعدلما مازــلإب مــكلحا - خــسفلا وأ شرلإا عــم كاــسملإا  . ةداــيزلا بــلط نــع رــظنلا فرــصو نــمثلا  . نيعادتلما ىوقأ بناج في عرشت نميلا   ةفوصوم ةرايــس اهيلع ىــعدلما ةكرــشلا نــم ىرتــشا هــنأب يــعدلما ىــعدا   في روــكذم اــهيف يــعنصم بــيع رــهظ اهمدختــسا نأ دــعبو هاوــعد في   هــلادبإب اــهيلع ىــعدلما ةكرــشلا ىــلع مــكلحا بــلط اذــل ، ىوــعدلا   لــيكو رــكنأ - ةــميقلا ةداــعإب اــهمازلإ كــلذ رذــعت اذإو ةــلثامم ةرايــس   ةرايــسلا في يــعنصم بــيع دوــجو نــم ىوــعدلا في ءاــج اــم اــهيلع ىــعدلما   مازــتللإا مدــع نــع تجاــن يــعدلما ىوــعد في روــكذلما لــطعلا نأــب عــفدو   صــحف تم - ماــظتناب كرــلمحا تــيز رــييغت مدــعب ةرايــسلا ةماــس دــعاوقب   صيخــشتلا في أــطلخا بــيعلا ببــس نأ اوأرــف ءارــبلخا لــبق نــم ةرايــسلا   ةــمكلمحا تررــق -تــيزلا رــييغت في ماــظتنلإا مدــع ســيلو ةكرــشلا نــم  27"""

# Reshape the text
reshaped_text = arabic_reshaper.reshape(text)

# Correct the direction
bidi_text = get_display(reshaped_text)

print(bidi_text)


27  ﻢــﻧ ﻼﺸــﺮﻛﺓ ﻮﻠﻴــﺳ ﻊــﺪﻣ ﺍﻺﻨﺘﻈــﺎﻣ ﻲﻓ ﺖﻐﻴﻴــﺭ ﻻﺰﻴــﺗ- ﻖــﺭﺮﺗ ﺎﺤﻤﻠﻜﻤــﺓ   ﻼﺴــﻳﺍﺭﺓ ﻢــﻧ ﻖﺒــﻟ ﺎﺨﻠﺒــﺭﺍﺀ ﻒــﺭﺃﻭﺍ ﺄﻧ ﺲــﺒﺑ ﻼﻌﻴــﺑ ﺎﺨﻠﻄــﺃ ﻲﻓ ﻼﺘﺸــﺨﻴﺻ   ﺐﻗﻭﺎﻌــﺩ ﺲــﺎﻣﺓ ﻼﺴــﻳﺍﺭﺓ ﺐﻌــﺪﻣ ﺖﻐﻴﻴــﺭ ﺰﻴــﺗ ﺎﺤﻤﻠــﺮﻛ ﺏﺎﻨﺘﻈــﺎﻣ - ﻢﺗ ﻒﺤــﺻ   ﻭﺪﻔــﻋ ﺐــﺄﻧ ﻼﻌﻄــﻟ ﺎﻤﻟﺬﻜــﻭﺭ ﻲﻓ ﺪﻌــﻮﯨ ﺎﻤﻟﺪﻌــﻳ ﻦــﺎﺠﺗ ﻊــﻧ ﻊــﺪﻣ ﺍﻺﻠﺘــﺯﺎﻣ   ﺎﻤﻟﺪﻌــﯨ ﻊﻠﻴﻬــﺍ ﻢــﺍ ﺞــﺍﺀ ﻲﻓ ﻻﺪﻌــﻮﯨ ﻢــﻧ ﻮﺠــﻭﺩ ﻊﻴــﺑ ﻢﺼﻨﻌــﻳ ﻲﻓ ﻼﺴــﻳﺍﺭﺓ   ﺲــﻳﺍﺭﺓ ﻢﻣﺎﺜﻠــﺓ ﻭﺇﺫﺍ ﺖﻌــﺫﺭ ﺬﻠــﻛ ﻹﺯﺎﻤﻬــﺍ ﺏﺈﻌــﺍﺩﺓ ﻼﻘﻴﻤــﺓ - ﺄﻨﻜــﺭ ﻮﻜﻴــﻟ   ﻻﺪﻌــﻮﯨ ، ﻞــﺫﺍ ﻂﻠــﺑ ﺎﺤﻠﻜــﻣ ﻊﻠــﯨ ﻼﺸــﺮﻛﺓ ﺎﻤﻟﺪﻌــﯨ ﻊﻠﻴﻬــﺍ ﺏﺈﺑﺩﻼــﻫ   ﻲﻓ ﺪﻌــﻭﺎﻫ ﻮﺒﻌــﺩ ﺄﻧ ﺎﺴــﺘﺧﺪﻤﻫﺍ ﻆﻬــﺭ ﻊﻴــﺑ ﻢﺼﻨﻌــﻳ ﻒﻴﻬــﺍ ﻡﺬﻜــﻭﺭ ﻲﻓ   ﺍﺪﻌــﯨ ﺎﻤﻟﺪﻌــﻳ ﺏﺄﻨــﻫ ﺎﺸــﺗﺮﯨ ﻢــﻧ ﻼﺸــﺮﻛﺓ ﺎﻤﻟﺪﻌــﯨ ﻊﻠﻴﻫﺍ ﺲــﻳﺍﺭﺓ ﻡﻮﺻﻮﻓﺓ   ﻼﻴﻤﻧ ﺖﺷﺮﻋ ﻲﻓ ﺝﺎﻨﺑ ﺄﻗﻮﯨ ﺎﻤﻠﺗﺩﺎﻌﻴﻧ .  ﻼﺜﻤــﻧ ﻮﺼــﺮﻓ ﻼﻨﻈــﺭ ﻊــﻧ ﻂﻠــﺑ ﻻﺰﻴــﺍﺩﺓ .  ﺍﻺﻤﺴــﺎﻛ ﻢــﻋ ﺍﻹﺮﺷ ﺃﻭ ﻼﻔﺴــﺧ - ﺎﺤﻠﻜــﻣ ﺏﻺــﺯﺎﻣ ﺎﻤﻟﺪﻌــﯨ ﻊﻠﻴــﻫ ﺖﺴــﻠﻴﻣ   ﺎﻤﻟﺪﻌــﯨ ﻢﻴــﻧ ﺍﻼﺴــﺘﻈﻫﺍﺭ - ﻊﻨــﺩ ﻮﺠــﻭﺩ ﻼﻌﻴــﺑ ﻒﻠﻠﻤﺸــﺗﺮﯨ ﺎﺨﻠﻴــﺍﺭ ﺐــﻧ   ﻖﺴــﻣ ﺎﺨﻠﺒــﺭﺍﺀ - ﻼﻴﻤــﻧ ﺖﺸــﺮﻋ ﻲﻓ ﺝﺎﻨــﺑ ﺄﻘــﻮﯨ ﺎﻤﻠﺗﺩﺎﻌﻴــﻧ - ﺢﻠــﻓ   ﺐﻴــﻋ ﺲــﻳﺍﺭ

In [11]:
import arabic_reshaper
from bidi.algorithm import get_display

# Your extracted text


# Split the text into words and reverse the order
words = text.split()
reversed_words = words[::-1]
reversed_text = ' '.join(reversed_words)

# Reshape and display the text
reshaped_text = arabic_reshaper.reshape(reversed_text)
bidi_text = get_display(reshaped_text)

print(bidi_text)


ﻩ:1433/5/17ﻪـ ﺕﺍﺮﻴﺧ 33252287 : ﻙ ﻼﺻ ﻡ ﺮﻗ ﻢﺤﻜﻣﺓ ﻢﻧ ﻼﺘﺻﺪﻴﻗ ﻕﺭﺍﺭ ﺮﻘﻣ 960 ﻻﺪﻋﻮﯨ: ﻡ ﺮﻗ 1434/1/6ﻪـ ﺕﺍﺮﻴﺨﻫ: :344533 ﺍﻼﺴﺘﺌﻧﺎﻓ ﻁﺮﻴــﻗ ﻊــﻧ ﻼﻌﻴــﺑ ﺖﻗﺪﻴــﺭ - ﻼﻌﻴــﺑ ﻝﻮﺠــﻭﺩ ﻼﺒﻴــﻋ ﻒﺴــﺧ - ﺲــﻳﺍﺭﺓ ﺐﻴــﻋ ﺢﻠــﻓ - ﺎﻤﻠﺗﺩﺎﻌﻴــﻧ ﺄﻘــﻮﯨ ﺝﺎﻨــﺑ ﻲﻓ ﺖﺸــﺮﻋ ﻼﻴﻤــﻧ - ﺎﺨﻠﺒــﺭﺍﺀ ﻖﺴــﻣ ﺐــﻧ ﺎﺨﻠﻴــﺍﺭ ﻒﻠﻠﻤﺸــﺗﺮﯨ ﻼﻌﻴــﺑ ﻮﺠــﻭﺩ ﻊﻨــﺩ - ﺍﻼﺴــﺘﻈﻫﺍﺭ ﻢﻴــﻧ ﺎﻤﻟﺪﻌــﯨ ﺖﺴــﻠﻴﻣ ﻊﻠﻴــﻫ ﺎﻤﻟﺪﻌــﯨ ﺏﻺــﺯﺎﻣ ﺎﺤﻠﻜــﻣ - ﻼﻔﺴــﺧ ﺃﻭ ﺍﻹﺮﺷ ﻢــﻋ ﺍﻺﻤﺴــﺎﻛ . ﻻﺰﻴــﺍﺩﺓ ﻂﻠــﺑ ﻊــﻧ ﻼﻨﻈــﺭ ﻮﺼــﺮﻓ ﻼﺜﻤــﻧ . ﺎﻤﻠﺗﺩﺎﻌﻴﻧ ﺄﻗﻮﯨ ﺝﺎﻨﺑ ﻲﻓ ﺖﺷﺮﻋ ﻼﻴﻤﻧ ﻡﻮﺻﻮﻓﺓ ﺲــﻳﺍﺭﺓ ﻊﻠﻴﻫﺍ ﺎﻤﻟﺪﻌــﯨ ﻼﺸــﺮﻛﺓ ﻢــﻧ ﺎﺸــﺗﺮﯨ ﺏﺄﻨــﻫ ﺎﻤﻟﺪﻌــﻳ ﺍﺪﻌــﯨ ﻲﻓ ﻡﺬﻜــﻭﺭ ﻒﻴﻬــﺍ ﻢﺼﻨﻌــﻳ ﻊﻴــﺑ ﻆﻬــﺭ ﺎﺴــﺘﺧﺪﻤﻫﺍ ﺄﻧ ﻮﺒﻌــﺩ ﺪﻌــﻭﺎﻫ ﻲﻓ ﺏﺈﺑﺩﻼــﻫ ﻊﻠﻴﻬــﺍ ﺎﻤﻟﺪﻌــﯨ ﻼﺸــﺮﻛﺓ ﻊﻠــﯨ ﺎﺤﻠﻜــﻣ ﻂﻠــﺑ ﻞــﺫﺍ ، ﻻﺪﻌــﻮﯨ ﻮﻜﻴــﻟ ﺄﻨﻜــﺭ - ﻼﻘﻴﻤــﺓ ﺏﺈﻌــﺍﺩﺓ ﻹﺯﺎﻤﻬــﺍ ﺬﻠــﻛ ﺖﻌــﺫﺭ ﻭﺇﺫﺍ ﻢﻣﺎﺜﻠــﺓ ﺲــﻳﺍﺭﺓ ﻼﺴــﻳﺍﺭﺓ ﻲﻓ ﻢﺼﻨﻌــﻳ ﻊﻴــﺑ ﻮﺠــﻭﺩ ﻢــﻧ ﻻﺪﻌــﻮﯨ ﻲﻓ ﺞــﺍﺀ ﻢــﺍ ﻊﻠﻴﻬــﺍ ﺎﻤﻟﺪﻌــﯨ ﺍﻺﻠﺘــﺯﺎﻣ ﻊــﺪﻣ ﻊــﻧ ﻦــﺎﺠﺗ ﺎﻤﻟﺪﻌــﻳ ﺪﻌــﻮﯨ ﻲﻓ ﺎﻤﻟﺬﻜــﻭﺭ ﻼﻌﻄــﻟ ﺐــﺄﻧ ﻭﺪﻔــﻋ ﻒﺤــﺻ ﻢﺗ - ﺏﺎﻨﺘﻈــﺎﻣ ﺎﺤﻤﻠــﺮﻛ ﺰﻴــﺗ ﺖﻐﻴﻴــﺭ ﺐﻌــﺪﻣ ﻼﺴــﻳﺍﺭﺓ ﺲــﺎﻣﺓ ﺐﻗﻭﺎﻌـ